<a href="https://colab.research.google.com/github/Huxwell/ColabNNs/blob/main/cats_n_dogs.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# torch.__version__
# !pip uninstall torch -y
# !pip install torch==1.11.0

In [ ]:
!nvidia-smi

In [ ]:
import torch.nn as nn
import torch
import torch.optim as optim
import torch.nn.functional as F
from torchvision import datasets, models, transforms
from torch.utils.data import DataLoader, Dataset
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import os
import zipfile
import glob
from tqdm import tqdm
from PIL import Image
import random # do random.seed(13) before every shuffle. order of shuffle() execution changes results order.

Notes:
- I have no labels for test here, so I am dropping 'test.zip' related code. I can split train into train, val, test; in fact I don't want to have a lot of examples for train set.
- The sets are almost balanced, accuracy is ok here

In [ ]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'

torch.manual_seed(13)
if device =='cuda':
    torch.cuda.manual_seed_all(13)

In [ ]:
dataset_dir = '/content/drive/MyDrive/cats_n_dogs_unsure/dogs-vs-cats-redux-kernels-edition'
subset_bad_crops_path = '/content/drive/MyDrive/cats_n_dogs_unsure/cat_dogs_crops_from_test_20k_filtered.zip'
subset_multi_path = '/content/drive/MyDrive/cat_dogs_8_contats_50_50_from_test.zip'
os.listdir(dataset_dir)

In [ ]:
os.makedirs('local_data/', exist_ok=True)
train_dir = 'local_data/train'

In [ ]:
with zipfile.ZipFile(os.path.join(dataset_dir, 'train.zip')) as train_zip:
    train_zip.extractall('local_data')

with zipfile.ZipFile(subset_bad_crops_path) as bad_crops_zip:
    bad_crops_zip.extractall('local_data')

with zipfile.ZipFile(subset_multi_path) as multi_zip:
    multi_zip.extractall('local_data')

In [ ]:
os.listdir(train_dir)[:5]
os.listdir('/content/local_data/cat_dogs_8_contats_50_50_from_test')[:5]

In [ ]:
cats_list = sorted(glob.glob(os.path.join(train_dir,'cat*.jpg')))
dogs_list = sorted(glob.glob(os.path.join(train_dir,'dog*.jpg')))
bad_cats_generated_from_test_list = sorted(glob.glob(os.path.join("local_data/cat_dogs_crops_from_test_20k_filtered/filtered_cats",'cat*.jpg')))
bad_dogs_generated_from_test_list = sorted(glob.glob(os.path.join("local_data/cat_dogs_crops_from_test_20k_filtered/filtered_dogs",'dog*.jpg')))
concat_8_50_50_list = sorted(glob.glob(os.path.join("local_data/cat_dogs_8_contats_50_50_from_test/",'cat_dog_50_50_8concats_*_224x224resize.jpg')))

print(concat_8_50_50_list[:5])
print(len(concat_8_50_50_list))

print(len(bad_cats_generated_from_test_list))
print(len(bad_dogs_generated_from_test_list))
print(len(cats_list))
print(len(dogs_list))
random.seed(13)
random.shuffle(cats_list)
random.seed(13) #multiple seed() executions are on purpose.
random.shuffle(dogs_list)
print(cats_list[:3])
print(dogs_list[:3])
# some sanity check to make sure no uncontrolled randomness beyond this point
assert cats_list[2] == 'local_data/train/cat.801.jpg'
assert dogs_list[2] == 'local_data/train/dog.801.jpg'
assert len(cats_list) == len(dogs_list)


random.seed(13)
random.shuffle(bad_cats_generated_from_test_list)
random.seed(13)
random.shuffle(bad_dogs_generated_from_test_list)
print(bad_cats_generated_from_test_list[:3])
print(bad_dogs_generated_from_test_list[:3])
assert bad_cats_generated_from_test_list[1] == 'local_data/cat_dogs_crops_from_test_20k_filtered/filtered_cats/cat.5232_crop_e_2.jpg'
assert bad_dogs_generated_from_test_list[1] == 'local_data/cat_dogs_crops_from_test_20k_filtered/filtered_dogs/dog.3799_crop_e_2.jpg'

In [ ]:
full_train_list = cats_list[:7500] + dogs_list[:7500]
# subset_train_list = cats_list[:500] + dogs_list[:500]
# subset_train_list = cats_list[:1000] + dogs_list[:1000]
# subset_train_list = cats_list[:2000] + dogs_list[:2000]
# subset_train_list = cats_list[:250] + dogs_list[:250]
# subset_train_list = cats_list[:150] + dogs_list[:150]
# subset_train_list = cats_list[:250] + dogs_list[:250] + bad_cats_generated_from_test_list[:100] + bad_dogs_generated_from_test_list[:100]
# subset_train_list = cats_list[:500] + dogs_list[:500] + bad_cats_generated_from_test_list[:200] + bad_dogs_generated_from_test_list[:200]

# subset_train_list = cats_list[:50] + dogs_list[:50] + bad_cats_generated_from_test_list[:20] + bad_dogs_generated_from_test_list[:20]

# subset_train_list = cats_list[:2000] + dogs_list[:2000] + bad_cats_generated_from_test_list[:800] + bad_dogs_generated_from_test_list[:800]
# subset_train_list = cats_list[:1000] + dogs_list[:1000] + bad_cats_generated_from_test_list[:400] + bad_dogs_generated_from_test_list[:400]
# subset_train_list = cats_list[:1000] + dogs_list[:1000] + concat_8_50_50_list[:400]

# subset_train_list = cats_list[:1000] + dogs_list[:1000] + concat_8_50_50_list[:400]
# subset_train_list = cats_list[:2000] + dogs_list[:2000] + concat_8_50_50_list[:800]

# subset_train_list = cats_list[:500] + dogs_list[:500] + concat_8_50_50_list[:400]
# subset_train_list = cats_list[:500] + dogs_list[:500] + concat_8_50_50_list[:200]
# subset_train_list = cats_list[:2000] + dogs_list[:2000] + concat_8_50_50_list[:1600]

# subset_train_list = cats_list[:50] + dogs_list[:50] + concat_8_50_50_list[:40]

# subset_train_list = cats_list[:250] + dogs_list[:250] + concat_8_50_50_list[:200]
subset_train_list = cats_list[:250] + dogs_list[:250] + concat_8_50_50_list[:100]


val_list = cats_list[7500:10000] + dogs_list[7500:10000]
test_list = cats_list[10000:] + dogs_list[10000:]
print(len(subset_train_list),len(full_train_list), len(val_list), len(test_list))

random.seed(13)
random.shuffle(full_train_list)
random.seed(13)
random.shuffle(subset_train_list)
random.seed(13)
random.shuffle(val_list)
random.seed(13)
random.shuffle(test_list)
print("full",full_train_list[:9])
print("subset",subset_train_list[:9])
print("val",val_list[:9])
print("test",test_list[:9])
assert full_train_list[4] == 'local_data/train/cat.1612.jpg'
# assert subset_train_list[4] == 'local_data/cat_dogs_crops_from_test_20k_filtered/filtered_cats/cat.2922_crop_e_0.jpg' #500 + 200 bad crop imgs train
# assert subset_train_list[1] == 'local_data/cat_dogs_crops_from_test_20k_filtered/filtered_cats/cat.4044_crop_e_3.jpg' #1000 + 400 bad crop imgs train
# assert subset_train_list[3] == 'local_data/cat_dogs_crops_from_test_20k_filtered/filtered_dogs/dog.4554_crop_e_2.jpg' #100 + 40 bad crop imgs train
# assert subset_train_list[0] == 'local_data/cat_dogs_crops_from_test_20k_filtered/filtered_dogs/dog.501_crop_e_0.jpg' #2000 + 800 bad crop imgs train
# assert subset_train_list[3] == 'local_data/cat_dogs_8_contats_50_50_from_test/cat_dog_50_50_8concats_1123_224x224resize.jpg' #2000 + 400 0.5 0.5


# assert subset_train_list[1] == 'local_data/cat_dogs_8_contats_50_50_from_test/cat_dog_50_50_8concats_1103_224x224resize.jpg' #1000 + 400 0.5 0.5
# assert subset_train_list[1] == 'local_data/cat_dogs_8_contats_50_50_from_test/cat_dog_50_50_8concats_1859_224x224resize.jpg' #4000 + 1600 0.5 0.5

# assert subset_train_list[3] == 'local_data/cat_dogs_8_contats_50_50_from_test/cat_dog_50_50_8concats_1032_224x224resize.jpg' #100 + 40 0.5 0.5
# assert subset_train_list[2] == 'local_data/cat_dogs_8_contats_50_50_from_test/cat_dog_50_50_8concats_1123_224x224resize.jpg' #500 + 200 0.5 0.5

assert subset_train_list[4] == 'local_data/cat_dogs_8_contats_50_50_from_test/cat_dog_50_50_8concats_1067_224x224resize.jpg' #500 + 100 0.5 0.5


# assert subset_train_list[3] == 'local_data/cat_dogs_8_contats_50_50_from_test/cat_dog_50_50_8concats_1007_224x224resize.jpg' #1000 + 200 0.5 0.5


# assert subset_train_list[1] == 'local_data/cat_dogs_crops_from_test_20k_filtered/filtered_dogs/dog.4650_crop_e_0.jpg' #4000 + 1600 bad crop imgs train

# assert subset_train_list[4] == 'local_data/train/cat.1787.jpg' #500imgs train
# assert subset_train_list[4] == 'local_data/train/cat.5360.jpg' #4000 imgs train
# assert subset_train_list[4] == 'local_data/train/dog.445.jpg' # 1000 imgs train
# assert subset_train_list[4] == 'local_data/train/dog.7661.jpg' # 300 imgs
# assert subset_train_list[4] == 'local_data/train/cat.9914.jpg' #100 imgs
# assert subset_train_list[4] == 'local_data/train/dog.11151.jpg'#2000imgs train
assert val_list[4] == 'local_data/train/dog.12023.jpg'
assert test_list[4] == 'local_data/train/dog.6485.jpg'

In [ ]:
train_list = subset_train_list # 5 batches # or 20 batches
# train_list = full_train_list

In [ ]:
np.random.seed(13131313)
random_idx = np.random.randint(1,len(train_list),size=10)
print(random_idx)
fig = plt.figure(figsize=(20., 10.))

for i, img_idx in enumerate(random_idx):
    ax = fig.add_subplot(2,5,i+1)
    img = Image.open(train_list[img_idx])
    plt.imshow(img)
    i+=1

plt.axis('off')
plt.show()

# Data Augmentation

In [ ]:
train_transforms =  transforms.Compose([ #these params work well for cats & dogs.:
  # transforms.Resize((224, 224)),
  transforms.RandomResizedCrop(224,scale=(0.9, 1.0)), 
  transforms.RandomHorizontalFlip(),
  transforms.ColorJitter(brightness=.4, hue=.15),
  transforms.ToTensor(),
])

val_transforms = transforms.Compose([
  transforms.Resize((224, 224)),
  # transforms.RandomResizedCrop(224),
  # transforms.RandomHorizontalFlip(),
  transforms.ToTensor(),
])

test_transforms = transforms.Compose([   
  transforms.Resize((224, 224)),
  # transforms.RandomResizedCrop(224),
  # transforms.RandomHorizontalFlip(),
  transforms.ToTensor()
])

In [ ]:
def get_hard_label(img_path):
  label = img_path.split('/')[-1].split('.')[0]
  if label == 'dog':
    return 1
  elif label == 'cat':
    return 0

def get_soft_label_inbalanced(img_path):
  label = img_path.split('/')[-1].split('.')[0]
  if label == 'dog':
      if "crop" in img_path:
        return torch.FloatTensor([0.7,0.3])
      else:
        return torch.FloatTensor([1.0,0.0])
  elif label == 'cat':
      if "crop" in img_path:
        return torch.FloatTensor([0.3,0.7])
      else:
        return torch.FloatTensor([0.0,1.0])

def get_soft_label_balanced(img_path):
  label = img_path.split('/')[-1].split('.')[0]
  if "cat" in label and "dog" in label and "concat" in label:
      return torch.FloatTensor([0.5,0.5])
  elif label == 'dog':
      return torch.FloatTensor([1.0,0.0])
  elif label == 'cat':
      return torch.FloatTensor([0.0,1.0])

In [ ]:
class dataset(torch.utils.data.Dataset):
    def __init__(self,file_list,transform=None):
        self.file_list = file_list
        self.transform = transform
        
    #dataset length
    def __len__(self):
        self.filelength = len(self.file_list)
        return self.filelength
    
    def __getitem__(self,idx):
        img_path = self.file_list[idx]
        img = Image.open(img_path)
        img_transformed = self.transform(img)
        
        # print("===========")
        # print("img path", img_path)
        label = get_soft_label_balanced(img_path)

        # print("label", label)
            
        return img_transformed,label

In [ ]:
## Visualization of Image Classification 
import torchvision
def visualize_classification(loader_iter, nrofItems = 9, pad = 0) -> None:

  #Iterate through the data loader
  imgTensor, labels = next(loader_iter)
  labels = labels.tolist()

  # Generate image grid
  grid = torchvision.utils.make_grid(imgTensor[:nrofItems], padding = pad, nrow=nrofItems)

  # Permute the axis as numpy expects image of shape (H x W x C) 
  grid = grid.permute(1, 2, 0)
  
  # Set up plot config
  plt.figure(figsize=(8, 2), dpi=300)
  plt.axis('off')

  # Plot Image Grid
  plt.imshow(grid)
  
  # # Plot the image titles
  # fact = 1 + (nrofItems)/100
  # rng = np.linspace(1/(fact*nrofItems), 1 - 1/(fact*nrofItems) , num = nrofItems)
  # for idx, val in enumerate(rng):
  #   plt.figtext(val, 0.85, labels[idx], fontsize=8)

  # Show the plot
  plt.show()

In [ ]:
batch_size = 100 # we will use mini-batch method
# batch_size = 4 # 10 TODO change this after going back to GPU! this is because notebook crashes when whole RAM is used # using 1 because of soft softmax bug/opti in pytorch

In [ ]:
train_data = dataset(train_list, transform=train_transforms)
val_data = dataset(val_list, transform=val_transforms)
test_data = dataset(test_list, transform=test_transforms)
visualise_train_dataset = dataset(train_list, transform=train_transforms)

train_loader = torch.utils.data.DataLoader(dataset = train_data, batch_size=batch_size, shuffle=False )
val_loader = torch.utils.data.DataLoader(dataset = val_data, batch_size=batch_size, shuffle=False)
test_loader = torch.utils.data.DataLoader(dataset = val_data, batch_size=batch_size, shuffle=False)
vis_train_loader = torch.utils.data.DataLoader(dataset = train_data, batch_size=10, shuffle=False)

In [ ]:
# this cell is here only because the Author apprently didn't understand how Dataset and Dataloaders behave - are they iterable, are they generators? - well they are hybrid. - , and wanted to see what happens
# all but one cell can be commented
# TODO: you don't need a separate loader. each iteration goes from the beginning
# but its interesting that different augmentations get generated.
iterator = iter(vis_train_loader)
visualize_classification(iterator)
visualize_classification(iterator)
visualize_classification(iterator)
visualize_classification(iter(vis_train_loader))
visualize_classification(iter(vis_train_loader))
visualize_classification(iter(vis_train_loader))

In [ ]:
print(len(train_data), len(train_loader), len(test_loader))

In [ ]:
print(sum(1 for filename in train_data.file_list if 'cat' in filename))
print(sum(1 for filename in train_data.file_list if 'dog' in filename))
print(sum(1 for filename in val_data.file_list if 'cat' in filename))
print(sum(1 for filename in val_data.file_list if 'dog' in filename))
print(sum(1 for filename in test_data.file_list if 'cat' in filename))
print(sum(1 for filename in test_data.file_list if 'dog' in filename))

In [ ]:
#check our images shape
train_data[0][0].shape

In [ ]:
# activations = torch.FloatTensor([[-0.3139, -0.0486],
# [-0.0510,  0.0470],
# [ 0.0963,  0.0143],
# [-0.2151, -0.0576]])

# targets_float = torch.FloatTensor([1.0, 1.0, 0.0, 0.5])
# targets_long = torch.LongTensor([1, 1, 0, 0])
# targets_float = torch.FloatTensor([[t, 1-t] for t in targets_float])
# print(targets_float)
# print(targets_float.shape)
# crit = nn.CrossEntropyLoss()
# crit(activations,targets_long) # tensor(0.6606)
# crit(activations,targets_float) 
# # return torch._C._nn.cross_entropy_loss(input, target, weight, _Reduction.get_enum(reduction), ignore_index, label_smoothing)
# # RuntimeError: expected scalar type Long but found Float

In [ ]:
class Experiment:
  epochs = 100 #10
  min_val_loss = float('inf')
  epochs_since_min_loss = 0
  patience = 7
  final_scores = {}  

  def __init__(self):
    self.model = torch.hub.load('pytorch/vision:v0.10.0', 'mobilenet_v2', pretrained=False)
    self.model.classifier[-1] = nn.Linear(in_features=1280, out_features=2, bias=True)
    self.model = self.model.to(device).train()
    self.optimizer = optim.Adam(params = self.model.parameters(),lr=0.001)
    self.criterion = nn.CrossEntropyLoss()

  def eval_model(self, loader: torch.utils.data.dataloader.DataLoader, model: nn.Module, description: str="Evaluation:") -> tuple: #[torch.Tensor, torch.Tensor] detailed tuple types only in Python >3.9, colab has 3.8.16
    epoch_accuracy=0
    epoch_loss =0
    with torch.no_grad():
      with tqdm(loader, unit="iteration") as tqdm_wrapped_loader:
        tqdm_wrapped_loader.set_description(description)
        for data, label in tqdm_wrapped_loader:
            data = data.to(device)
            label = label.to(device)

            output = model(data)
            # print(output, label)
            # loss = run.criterion(output,label.float())
            loss = run.criterion(output,label)


            # acc = ((output.argmax(dim=1) == label).float().mean())
            acc = ((output.argmax(dim=1) == label.argmax(dim=1)).float().mean())

            epoch_accuracy += acc/ len(loader)
            epoch_loss += loss/ len(loader)
            tqdm_wrapped_loader.set_postfix(epoch_accuracy=epoch_accuracy, loss=epoch_loss)
    return epoch_accuracy, epoch_loss

  def train(self):
      for epoch in range(self.epochs):
          print("\n","="*30,"\n")
          epoch_loss = 0
          epoch_accuracy = 0
          
          with tqdm(train_loader, unit="iteration") as tepoch:
            
            for data, label in tepoch:
                tepoch.set_description(f"Training epoch {epoch}")
                data = data.to(device)
                label = label.to(device)
                
                output = run.model(data)
                # print("output train", output, output.shape)

                # print("target train", label)
                loss = run.criterion(output, label)
                # loss = run.criterion(output, label.float())
                
                run.optimizer.zero_grad()
                loss.backward()
                run.optimizer.step()
                
                acc = ((output.argmax(dim=1) == label.argmax(dim=1)).float().mean())

                # acc = ((output.argmax(dim=1) == label).float().mean())

                epoch_accuracy += acc/len(train_loader)
                epoch_loss += loss/len(train_loader)
                tepoch.set_postfix(loss=loss.item(), accuracy=100. * epoch_accuracy)
            print('Epoch : {}, train accuracy : {}, train loss : {}'.format(epoch, epoch_accuracy,epoch_loss))

          epoch_val_accuracy,epoch_val_loss = self.eval_model(val_loader, run.model, f"Validation after epoch {epoch}")
          print('Epoch : {}, val_accuracy : {}, val_loss : {}'.format(epoch, epoch_val_accuracy,epoch_val_loss))
          scalar_val_loss = epoch_val_loss.item()
          if scalar_val_loss < self.min_val_loss:
            self.min_val_loss = scalar_val_loss 
            self.epochs_since_min_loss = 0 
            self.final_scores['epoch']=epoch
            self.final_scores['epoch_train_accuracy']=epoch_accuracy
            self.final_scores['epoch_train_loss']=epoch_loss
            self.final_scores['epoch_val_loss']=epoch_val_loss
            self.final_scores['epoch_val_accuracy']=epoch_val_accuracy


            print("New best model, min_val_loss:", self.min_val_loss)

            # epoch_test_accuracy,epoch_test_loss = self.eval_model(test_loader, model, f"Test after epoch {epoch}")
            # print('Epoch : {}, test_accuracy : {}, test_loss : {}'.format(epoch, epoch_test_accuracy,epoch_test_loss))



          else:
            self.epochs_since_min_loss+=1
            print("epochs_since_min_loss",self.epochs_since_min_loss)
          if self.epochs_since_min_loss > self.patience:
            print(f"Early stopping. \n\n Best model scores: {self.final_scores}")
            break

In [ ]:
output = torch.FloatTensor([[-0.1185,  0.7045],
        [ 0.3875, -0.1597],
        [-0.0238,  0.2865],
        [-0.0509,  0.2330]]) 

label = torch.FloatTensor([[1., 0.],
        [1., 0.],
        [0.3, 0.7],
        [1., 0.]])

print(output.argmax(dim=1))
print(label.argmax(dim=1))
print((output.argmax(dim=1) == label.argmax(dim=1)).float().mean())
# acc = ((output.argmax(dim=1) == label).float().mean())
# epoch_accuracy += acc/len(train_loader)

In [ ]:
run = Experiment()
run.train()

In [ ]:
print(f"Early stopping. \n\n Best model scores: {run.final_scores}")

In [ ]:
dog_probs = []
run.model.eval()
i=0
with torch.no_grad():
  for data, fileid in val_loader:
      i+=1
      if i>10:
        break
      data = data.to(device)
      preds = run.model(data)
      print(preds)
      preds_list = F.softmax(preds, dim=1)[:, 1].tolist() #https://stats.stackexchange.com/questions/542054/why-does-torchvision-models-resnet18-not-use-softmax
      dog_probs += list(zip(list(fileid), preds_list))

# Future experiments
1000 imgs set; or more intervals; saturate cheap experiments space.

no test
no random crop augmentation
# Results

batch=100, patience=7
============

2000 imgs train set,
transforms.RandomResizedCrop(224,scale=(0.6, 1.0)), 
transforms.RandomHorizontalFlip(),
transforms.ColorJitter(brightness=.4, hue=.15),
model = torch.hub.load('pytorch/vision:v0.10.0', 'mobilenet_v2', pretrained=False)

Best model scores: 
 {'epoch': 33, 'epoch_train_accuracy': tensor(0.8380, device='cuda:0'), 'epoch_train_loss': tensor(0.3546, device='cuda:0', grad_fn=<AddBackward0>), 'epoch_val_loss': tensor(0.3896, device='cuda:0'), 'epoch_val_accuracy': tensor(0.8188, device='cuda:0')}

Training epoch 33: 100%|██████████| 20/20 [00:25<00:00,  1.30s/iteration, accuracy=tensor(83.8000, device='cuda:0'), loss=0.395]
Epoch : 33, train accuracy : 0.8380000591278076, train loss : 0.3545871675014496
100%|██████████| 50/50 [00:25<00:00,  1.93iteration/s, epoch_val_accuracy=tensor(0.8188, device='cuda:0'), val_loss=tensor(0.4697, device='cuda:0')]
Epoch : 33, val_accuracy : 0.8188000321388245, val_loss : 0.3895597457885742
New best model, min_val_loss: 0.3895597457885742



================================================

500 imgs train set,
transforms.RandomResizedCrop(224,scale=(0.6, 1.0)), 
transforms.RandomHorizontalFlip(),
transforms.ColorJitter(brightness=.4, hue=.15),
model = torch.hub.load('pytorch/vision:v0.10.0', 'mobilenet_v2', pretrained=False)

Best model scores: {'epoch': 39, 'epoch_train_accuracy': tensor(0.7480, device='cuda:0'), 'epoch_train_loss': tensor(0.5132, device='cuda:0', grad_fn=<AddBackward0>), 'epoch_val_loss': tensor(0.5940, device='cuda:0'), 'epoch_val_accuracy': tensor(0.6928, device='cuda:0')}


Training epoch 39: 100%|██████████| 5/5 [00:06<00:00,  1.27s/iteration, accuracy=tensor(74.8000, device='cuda:0'), loss=0.504]
Epoch : 39, train accuracy : 0.7479999661445618, train loss : 0.5132189393043518
100%|██████████| 50/50 [00:30<00:00,  1.65iteration/s, epoch_val_accuracy=tensor(0.6928, device='cuda:0'), val_loss=tensor(0.6708, device='cuda:0')]
Epoch : 39, val_accuracy : 0.6927998661994934, val_loss : 0.5940383076667786
New best model, min_val_loss: 0.5940383076667786
100%|██████████| 50/50 [00:25<00:00,  1.93iteration/s, epoch_test_accuracy=tensor(0.6874, device='cuda:0'), test_loss=tensor(0.6752, device='cuda:0')]
Epoch : 39, test_accuracy : 0.6873999238014221, test_loss : 0.5957822799682617

================================================

500 imgs train set,
no transforms (only totensor)
fails with wrong shapes

================================================

500 imgs train set, only 224 resize
 transforms.Resize((224, 224)),


 Best model scores: {'epoch': 2, 'epoch_train_accuracy': tensor(0.6660, device='cuda:0'), 'epoch_train_loss': tensor(0.6178, device='cuda:0', grad_fn=<AddBackward0>), 'epoch_val_loss': tensor(0.6951, device='cuda:0'), 'epoch_val_accuracy': tensor(0.5774, device='cuda:0')}


 Training epoch 2: 100%|██████████| 5/5 [00:03<00:00,  1.28iteration/s, accuracy=tensor(66.6000, device='cuda:0'), loss=0.627]
Epoch : 2, train accuracy : 0.6660000085830688, train loss : 0.6177714467048645
100%|██████████| 50/50 [00:27<00:00,  1.85iteration/s, epoch_val_accuracy=tensor(0.5774, device='cuda:0'), val_loss=tensor(0.7848, device='cuda:0')]
Epoch : 2, val_accuracy : 0.5773999691009521, val_loss : 0.6951212882995605
New best model, min_val_loss: 0.6951212882995605
100%|██████████| 50/50 [00:25<00:00,  1.95iteration/s, epoch_test_accuracy=tensor(0.5704, device='cuda:0'), test_loss=tensor(0.7834, device='cuda:0')]
Epoch : 2, test_accuracy : 0.5703999996185303, test_loss : 0.6962302327156067

note: patience in my implementation (> patience) means 8 more epochs get executed.








========================================


full train set


Training epoch 40: 100%|██████████| 150/150 [03:28<00:00,  1.39s/iteration, accuracy=tensor(96.8067, device='cuda:0'), loss=0.0306]
Epoch : 40, train accuracy : 0.9680668115615845, train loss : 0.07920999079942703
100%|██████████| 50/50 [00:30<00:00,  1.65iteration/s, epoch_val_accuracy=tensor(0.9506, device='cuda:0'), val_loss=tensor(0.1377, device='cuda:0')]
Epoch : 40, val_accuracy : 0.9506001472473145, val_loss : 0.12937913835048676
New best model, min_val_loss: 0.12937913835048676
100%|██████████| 50/50 [00:30<00:00,  1.66iteration/s, epoch_test_accuracy=tensor(0.9522, device='cuda:0'), test_loss=tensor(0.1373, device='cuda:0')]
Epoch : 40, test_accuracy : 0.9522001147270203, test_loss : 0.12997546792030334

 Best model scores: {'epoch': 40, 'epoch_train_accuracy': tensor(0.9681, device='cuda:0'), 'epoch_train_loss': tensor(0.0792, device='cuda:0', grad_fn=<AddBackward0>), 'epoch_val_loss': tensor(0.1294, device='cuda:0'), 'epoch_val_accuracy': tensor(0.9506, device='cuda:0')}



 ============================== 

 4000 imgs
 

Training epoch 56: 100%|██████████| 40/40 [00:50<00:00,  1.26s/iteration, accuracy=tensor(95.7750, device='cuda:0'), loss=0.112]
Epoch : 56, train accuracy : 0.9577500224113464, train loss : 0.10647499561309814
Validation after epoch 56: 100%|██████████| 50/50 [00:25<00:00,  1.94iteration/s, epoch_accuracy=tensor(0.8970, device='cuda:0'), loss=tensor(0.2677, device='cuda:0')]Epoch : 56, val_accuracy : 0.8970000147819519, val_loss : 0.26771894097328186
epochs_since_min_loss 8
Early stopping. 

 Best model scores: {'epoch': 48, 'epoch_train_accuracy': tensor(0.9498, device='cuda:0'), 'epoch_train_loss': tensor(0.1290, device='cuda:0', grad_fn=<AddBackward0>), 'epoch_val_loss': tensor(0.2475, device='cuda:0'), 'epoch_val_accuracy': tensor(0.8988, device='cuda:0')}


  ============================== 
1000 imgs
Training epoch 46: 100%|██████████| 10/10 [00:12<00:00,  1.25s/iteration, accuracy=tensor(82.7000, device='cuda:0'), loss=0.423]
Epoch : 46, train accuracy : 0.8270000219345093, train loss : 0.3939513862133026
Validation after epoch 46: 100%|██████████| 50/50 [00:26<00:00,  1.92iteration/s, epoch_accuracy=tensor(0.7390, device='cuda:0'), loss=tensor(0.5494, device='cuda:0')]Epoch : 46, val_accuracy : 0.7390000224113464, val_loss : 0.5493948459625244
epochs_since_min_loss 8
Early stopping. 

 Best model scores: {'epoch': 38, 'epoch_train_accuracy': tensor(0.7650, device='cuda:0'), 'epoch_train_loss': tensor(0.4642, device='cuda:0', grad_fn=<AddBackward0>), 'epoch_val_loss': tensor(0.5249, device='cuda:0'), 'epoch_val_accuracy': tensor(0.7464, device='cuda:0')}

[29]
0s
print(f"Early stopping. \n\n Best model scores: {final_scores}")
Early stopping. 

=============================================

300 train imgs


 Best model scores: {'epoch': 36, 'epoch_train_accuracy': tensor(0.7300, device='cuda:0'), 'epoch_train_loss': tensor(0.5215, device='cuda:0', grad_fn=<AddBackward0>), 'epoch_val_loss': tensor(0.6209, device='cuda:0'), 'epoch_val_accuracy': tensor(0.6694, device='cuda:0')}


 Training epoch 36: 100%|██████████| 3/3 [00:03<00:00,  1.25s/iteration, accuracy=tensor(73., device='cuda:0'), loss=0.505]
Epoch : 36, train accuracy : 0.7300000190734863, train loss : 0.5214951038360596
Validation after epoch 36: 100%|██████████| 50/50 [00:25<00:00,  1.93iteration/s, epoch_accuracy=tensor(0.6694, device='cuda:0'), loss=tensor(0.6209, device='cuda:0')]
Epoch : 36, val_accuracy : 0.6693997979164124, val_loss : 0.6209338903427124
New best model, min_val_loss: 0.6209338903427124



===================================

100 imgs train



 Best model scores: {'epoch': 26, 'epoch_train_accuracy': tensor(0.6100, device='cuda:0'), 'epoch_train_loss': tensor(0.6548, device='cuda:0', grad_fn=<AddBackward0>), 'epoch_val_loss': tensor(0.6606, device='cuda:0'), 'epoch_val_accuracy': tensor(0.6008, device='cuda:0')}










 ========================


 1000 imgs train 
 no random crop


 Training epoch 11: 100%|██████████| 10/10 [00:13<00:00,  1.32s/iteration, accuracy=tensor(66.1000, device='cuda:0'), loss=0.598]
Epoch : 11, train accuracy : 0.6610000133514404, train loss : 0.6124292612075806
Validation after epoch 11: 100%|██████████| 50/50 [00:28<00:00,  1.78iteration/s, epoch_accuracy=tensor(0.6646, device='cuda:0'), loss=tensor(0.6104, device='cuda:0')]
Epoch : 11, val_accuracy : 0.6645999550819397, val_loss : 0.6103843450546265
New best model, min_val_loss: 0.6103843450546265


=======

4000 imgs no random crop

Best model scores: {'epoch': 21, 'epoch_train_accuracy': tensor(0.8915, device='cuda:0'), 'epoch_train_loss': tensor(0.2574, device='cuda:0', grad_fn=<AddBackward0>), 'epoch_val_loss': tensor(0.4274, device='cuda:0'), 'epoch_val_accuracy': tensor(0.8212, device='cuda:0')}



=======

4000 imgs random crop (0.9, 1.0)



 Best model scores: {'epoch': 38, 'epoch_train_accuracy': tensor(0.9482, device='cuda:0'), 'epoch_train_loss': tensor(0.1189, device='cuda:0', grad_fn=<AddBackward0>), 'epoch_val_loss': tensor(0.3618, device='cuda:0'), 'epoch_val_accuracy': tensor(0.8702, device='cuda:0')}


 =============

 100 (0.9,1.0)


 
 Best model scores: {'epoch': 18, 'epoch_train_accuracy': tensor(0.6400, device='cuda:0'), 'epoch_train_loss': tensor(0.6298, device='cuda:0', grad_fn=<AddBackward0>), 'epoch_val_loss': tensor(0.6644, device='cuda:0'), 'epoch_val_accuracy': tensor(0.6080, device='cuda:0')}

 99.9% sure it was actually 100 samples.


 ==================================

300 (0.9,1.0)
Best model scores: {'epoch': 13, 'epoch_train_accuracy': tensor(0.6600, device='cuda:0'), 'epoch_train_loss': tensor(0.6104, device='cuda:0', grad_fn=<AddBackward0>), 'epoch_val_loss': tensor(0.6303, device='cuda:0'), 'epoch_val_accuracy': tensor(0.6474, device='cuda:0')}

==================================


1000 (0.9, 1.0)

 Best model scores: {'epoch': 31, 'epoch_train_accuracy': tensor(0.7830, device='cuda:0'), 'epoch_train_loss': tensor(0.4721, device='cuda:0', grad_fn=<AddBackward0>), 'epoch_val_loss': tensor(0.5570, device='cuda:0'), 'epoch_val_accuracy': tensor(0.7226, device='cuda:0')}


 =========

2000 0.9-1.0

  Best model scores: {'epoch': 20, 'epoch_train_accuracy': tensor(0.7770, device='cuda:0'), 'epoch_train_loss': tensor(0.4572, device='cuda:0', grad_fn=<AddBackward0>), 'epoch_val_loss': tensor(0.5028, device='cuda:0'), 'epoch_val_accuracy': tensor(0.7600, device='cuda:0')}

==========

500 0.9-1.0

Training epoch 11: 100%|██████████| 5/5 [00:06<00:00,  1.29s/iteration, accuracy=tensor(67.2000, device='cuda:0'), loss=0.631]
Epoch : 11, train accuracy : 0.671999990940094, train loss : 0.623713493347168
Validation after epoch 11: 100%|██████████| 50/50 [00:26<00:00,  1.91iteration/s, epoch_accuracy=tensor(0.6492, device='cuda:0'), loss=tensor(0.6313, device='cuda:0')]
Epoch : 11, val_accuracy : 0.6492000222206116, val_loss : 0.6313157677650452
New best model, min_val_loss: 0.6313157677650452



==========

500 0.9-1.0 + 200 bad crops

Training epoch 19: 100%|██████████| 7/7 [00:08<00:00,  1.24s/iteration, accuracy=tensor(70.7143, device='cuda:0'), loss=0.569]

Epoch : 19, train accuracy : 0.7071428894996643, train loss : 0.5531335473060608

Validation after epoch 19: 100%|██████████| 50/50 [00:26<00:00,  1.90iteration/s, epoch_accuracy=tensor(0.6684, device='cuda:0'), loss=tensor(0.6105, device='cuda:0')]

Epoch : 19, val_accuracy : 0.6683999300003052, val_loss : 0.6104905605316162
New best model, min_val_loss: 0.6104905605316162

=============

1000 0.9-1.0 + 400 bad crops

Best model scores: {'epoch': 32, 'epoch_train_accuracy': tensor(0.8057, device='cuda:0'), 'epoch_train_loss': tensor(0.4207, device='cuda:0', grad_fn=<AddBackward0>), 'epoch_val_loss': tensor(0.5650, device='cuda:0'), 'epoch_val_accuracy': tensor(0.7372, device='cuda:0')}

Training epoch 32: 100%|██████████| 14/14 [00:18<00:00,  1.34s/iteration, accuracy=tensor(80.5714, device='cuda:0'), loss=0.399]

Epoch : 32, train accuracy : 0.8057142496109009, train loss : 0.42074477672576904

Validation after epoch 32: 100%|██████████| 50/50 [00:26<00:00,  1.90iteration/s, epoch_accuracy=tensor(0.7372, device='cuda:0'), loss=tensor(0.5650, device='cuda:0')]

Epoch : 32, val_accuracy : 0.7372000217437744, val_loss : 0.56500244140625
New best model, min_val_loss: 0.56500244140625

========================


100 0.9-1.0 + 40 bad crops

 Best model scores: {'epoch': 5, 'epoch_train_accuracy': tensor(0.5725, device='cuda:0'), 'epoch_train_loss': tensor(0.6900, device='cuda:0', grad_fn=<AddBackward0>), 'epoch_val_loss': tensor(0.6688, device='cuda:0'), 'epoch_val_accuracy': tensor(0.6106, device='cuda:0')}



==================

4000 + 1600 bad crops
 Best model scores: {'epoch': 32, 'epoch_train_accuracy': tensor(0.8988, device='cuda:0'), 'epoch_train_loss': tensor(0.2333, device='cuda:0', grad_fn=<AddBackward0>), 'epoch_val_loss': tensor(0.3286, device='cuda:0'), 'epoch_val_accuracy': tensor(0.8688, device='cuda:0')}



Training epoch 32: 100%|██████████| 56/56 [01:09<00:00,  1.24s/iteration, accuracy=tensor(89.8750, device='cuda:0'), loss=0.216]

Epoch : 32, train accuracy : 0.898750364780426, train loss : 0.23328110575675964

Validation after epoch 32: 100%|██████████| 50/50 [00:26<00:00,  1.92iteration/s, epoch_accuracy=tensor(0.8688, device='cuda:0'), loss=tensor(0.3286, device='cuda:0')]

Epoch : 32, val_accuracy : 0.8687999248504639, val_loss : 0.32859963178634644
New best model, min_val_loss: 0.32859963178634644


=================================

500 + 200 bad crops
results SHOULD be the same as before, since bad crops got 0.0 and 1.0 labels respectively


Training epoch 19: 100%|██████████| 7/7 [00:08<00:00,  1.21s/iteration, accuracy=tensor(69.0000, device='cuda:0'), loss=0.637]
Epoch : 19, train accuracy : 0.6899999380111694, train loss : 0.5906238555908203
Validation after epoch 19: 100%|██████████| 50/50 [00:26<00:00,  1.92iteration/s, epoch_accuracy=tensor(0.6570, device='cuda:0'), loss=tensor(0.6265, device='cuda:0')]
Epoch : 19, val_accuracy : 0.6569998860359192, val_loss : 0.6265230774879456
New best model, min_val_loss: 0.6265230774879456

===============================

500 + 200 bad crops, but 0.7 to 0.3 for bad crops

Training epoch 29: 100%|██████████| 7/7 [00:08<00:00,  1.24s/iteration, accuracy=tensor(76.7143, device='cuda:0'), loss=0.529]
Epoch : 29, train accuracy : 0.7671428918838501, train loss : 0.5189324617385864
Validation after epoch 29: 100%|██████████| 50/50 [00:25<00:00,  1.96iteration/s, epoch_accuracy=tensor(0.6740, device='cuda:0'), loss=tensor(0.6259, device='cuda:0')]
Epoch : 29, val_accuracy : 0.6739999651908875, val_loss : 0.625901997089386
New best model, min_val_loss: 0.625901997089386


==========================

4000 + 1600 bad crops, but 0.7 to 0.3 for bad crops


Training epoch 38: 100%|██████████| 56/56 [01:08<00:00,  1.22s/iteration, accuracy=tensor(89.8214, device='cuda:0'), loss=0.254]
Epoch : 38, train accuracy : 0.8982144594192505, train loss : 0.2613590359687805
Validation after epoch 38: 100%|██████████| 50/50 [00:25<00:00,  1.98iteration/s, epoch_accuracy=tensor(0.9016, device='cuda:0'), loss=tensor(0.2471, device='cuda:0')]
Epoch : 38, val_accuracy : 0.9015999436378479, val_loss : 0.24705414474010468
New best model, min_val_loss: 0.24705414474010468



======================



1000 + 400 bad crops, but 0.7 to 0.3 for bad crops


Training epoch 40: 100%|██████████| 14/14 [00:18<00:00,  1.30s/iteration, accuracy=tensor(84.0714, device='cuda:0'), loss=0.44]

Epoch : 40, train accuracy : 0.8407143354415894, train loss : 0.3932029604911804

Validation after epoch 40: 100%|██████████| 50/50 [00:27<00:00,  1.79iteration/s, epoch_accuracy=tensor(0.7522, device='cuda:0'), loss=tensor(0.5355, device='cuda:0')]

Epoch : 40, val_accuracy : 0.7521999478340149, val_loss : 0.5354558229446411
New best model, min_val_loss: 0.5354558229446411



Early stopping.  Best model scores: {'epoch': 40, 'epoch_train_accuracy': 0.8407, 'epoch_train_loss': 0.3932, 'epoch_val_loss': 0.5355, 'epoch_val_accuracy': 0.7522



======================================


100 + 40 bad crops, but 0.7 to 0.3 for bad crops

 Best model scores: {'epoch': 2, 'epoch_train_accuracy': tensor(0.5050, device='cuda:0'), 'epoch_train_loss': tensor(0.7095, device='cuda:0', grad_fn=<AddBackward0>), 'epoch_val_loss': tensor(0.6726, device='cuda:0'), 'epoch_val_accuracy': tensor(0.5798, device='cuda:0')}




=======================================


2000 + 800, 0.7/0.3

subset_train_list = cats_list[:1000] + dogs_list[:1000] + bad_cats_generated_from_test_list[:400] + bad_dogs_generated_from_test_list[:400]


Early stopping. 

 Best model scores: {'epoch': 34, 'epoch_train_accuracy': tensor(0.8429, device='cuda:0'), 'epoch_train_loss': tensor(0.3801, device='cuda:0', grad_fn=<AddBackward0>), 'epoch_val_loss': tensor(0.4131, device='cuda:0'), 'epoch_val_accuracy': tensor(0.8152, device='cuda:0')}


==================

2000 + 800 hard labels


subset_train_list = cats_list[:1000] + dogs_list[:1000] + bad_cats_generated_from_test_list[:400] + bad_dogs_generated_from_test_list[:400]  




Early stopping. 

 Best model scores: {'epoch': 29, 'epoch_train_accuracy': tensor(0.8254, device='cuda:0'), 'epoch_train_loss': tensor(0.3812, device='cuda:0', grad_fn=<AddBackward0>), 'epoch_val_loss': tensor(0.4917, device='cuda:0'), 'epoch_val_accuracy': tensor(0.7754, device='cuda:0')}



==============

2000 + 400 50/50 concats


Training epoch 28: 100%|██████████| 24/24 [00:29<00:00,  1.24s/iteration, accuracy=tensor(80.8333, device='cuda:0'), loss=0.314]
Epoch : 28, train accuracy : 0.8083333373069763, train loss : 0.38290396332740784
Validation after epoch 28: 100%|██████████| 50/50 [00:25<00:00,  1.99iteration/s, epoch_accuracy=tensor(0.8124, device='cuda:0'), loss=tensor(0.4261, device='cuda:0')]
Epoch : 28, val_accuracy : 0.8123999238014221, val_loss : 0.4260599613189697
New best model, min_val_loss: 0.4260599613189697


Early stopping. 

 Best model scores: {'epoch': 28, 'epoch_train_accuracy': tensor(0.8083, device='cuda:0'), 'epoch_train_loss': tensor(0.3829, device='cuda:0', grad_fn=<AddBackward0>), 'epoch_val_loss': tensor(0.4261, device='cuda:0'), 'epoch_val_accuracy': tensor(0.8124, device='cuda:0')}



=============

4000 + 800 0.5 0.5 concats

Training epoch 26: 100%|██████████| 48/48 [01:02<00:00,  1.31s/iteration, accuracy=tensor(85.8958, device='cuda:0'), loss=0.217]
Epoch : 26, train accuracy : 0.8589583039283752, train loss : 0.2704243063926697
Validation after epoch 26: 100%|██████████| 50/50 [00:28<00:00,  1.77iteration/s, epoch_accuracy=tensor(0.8670, device='cuda:0'), loss=tensor(0.3314, device='cuda:0')]
Epoch : 26, val_accuracy : 0.8670000433921814, val_loss : 0.33141279220581055
New best model, min_val_loss: 0.33141279220581055


==================


1000 + 200 0.5 0.5 concat


Training epoch 32: 100%|██████████| 12/12 [00:15<00:00,  1.32s/iteration, accuracy=tensor(76.7500, device='cuda:0'), loss=0.472]
Epoch : 32, train accuracy : 0.7674999833106995, train loss : 0.4808453619480133
Validation after epoch 32: 100%|██████████| 50/50 [00:28<00:00,  1.78iteration/s, epoch_accuracy=tensor(0.7368, device='cuda:0'), loss=tensor(0.5295, device='cuda:0')]
Epoch : 32, val_accuracy : 0.7367998957633972, val_loss : 0.5294669270515442
New best model, min_val_loss: 0.5294669270515442



===========================

1000 + 400 0.5 0.5 concat

Training epoch 29: 100%|██████████| 14/14 [00:17<00:00,  1.23s/iteration, accuracy=tensor(73.0000, device='cuda:0'), loss=0.572]
Epoch : 29, train accuracy : 0.7300000786781311, train loss : 0.5013352632522583
Validation after epoch 29: 100%|██████████| 50/50 [00:26<00:00,  1.87iteration/s, epoch_accuracy=tensor(0.7268, device='cuda:0'), loss=tensor(0.5353, device='cuda:0')]
Epoch : 29, val_accuracy : 0.7267997860908508, val_loss : 0.535283625125885
New best model, min_val_loss: 0.535283625125885


======================

4000 + 1600 0.5 0.5 concat


Training epoch 31: 100%|██████████| 56/56 [01:12<00:00,  1.29s/iteration, accuracy=tensor(82.5179, device='cuda:0'), loss=0.274]

Epoch : 31, train accuracy : 0.8251785635948181, train loss : 0.30084556341171265

Validation after epoch 31: 100%|██████████| 50/50 [00:28<00:00,  1.77iteration/s, epoch_accuracy=tensor(0.8682, device='cuda:0'), loss=tensor(0.3248, device='cuda:0')]

Epoch : 31, val_accuracy : 0.8682000041007996, val_loss : 0.3248395025730133
New best model, min_val_loss: 0.3248395025730133


===========================


100 + 40 0.5 0.5

Training epoch 13: 100%|██████████| 2/2 [00:01<00:00,  1.08iteration/s, accuracy=tensor(52.5000, device='cuda:0'), loss=0.655]

Epoch : 13, train accuracy : 0.5249999761581421, train loss : 0.6782363653182983

Validation after epoch 13: 100%|██████████| 50/50 [00:28<00:00,  1.75iteration/s, epoch_accuracy=tensor(0.5844, device='cuda:0'), loss=tensor(0.6757, device='cuda:0')]

Epoch : 13, val_accuracy : 0.584399938583374, val_loss : 0.6757243871688843
New best model, min_val_loss: 0.6757243871688843



====================


500 + 200 0.5 0.5


Training epoch 25: 100%|██████████| 7/7 [00:09<00:00,  1.34s/iteration, accuracy=tensor(65.7143, device='cuda:0'), loss=0.592]

Epoch : 25, train accuracy : 0.6571428775787354, train loss : 0.5981876850128174

Validation after epoch 25: 100%|██████████| 50/50 [00:28<00:00,  1.76iteration/s, epoch_accuracy=tensor(0.6582, device='cuda:0'), loss=tensor(0.6176, device='cuda:0')]

Epoch : 25, val_accuracy : 0.6581999659538269, val_loss : 0.6176449060440063
New best model, min_val_loss: 0.6176449060440063



=================

500 + 100 0.5 0.5

Training epoch 22: 100%|██████████| 6/6 [00:07<00:00,  1.27s/iteration, accuracy=tensor(65., device='cuda:0'), loss=0.606]

Epoch : 22, train accuracy : 0.6500000357627869, train loss : 0.6212294101715088

Validation after epoch 22: 100%|██████████| 50/50 [00:28<00:00,  1.74iteration/s, epoch_accuracy=tensor(0.6530, device='cuda:0'), loss=tensor(0.6255, device='cuda:0')]

Epoch : 22, val_accuracy : 0.6529998779296875, val_loss : 0.6254665851593018
New best model, min_val_loss: 0.6254665851593018
